In [1]:
import requests
from bs4 import BeautifulSoup
import json

In [17]:
data = []

def extract_text(element):
    #take both p and h2 tags
    if element.name == 'p' or (element.name == 'h2' and 'wp-block-heading' in element.get('class', [])):
        return element.text.replace('\n', ' ').strip()
    else:
        return None

for x in range(1, 4):
    response = requests.get(f"https://www.malaysiapropertyreviews.com/blog/page/{x}")
    soup = BeautifulSoup(response.content, "html.parser")
    entries = soup.find_all("h2", attrs={"class": "blog-entry-title entry-title"})

    for entry in entries:
        blog_url = entry.find("a").get("href")
        blog_title = entry.text.strip()

        page = requests.get(blog_url)
        blog_soup = BeautifulSoup(page.text, "html.parser")

        body = blog_soup.find("div", attrs={"class": "nv-content-wrap entry-content"})

        texts = [extract_text(tag) for tag in body.find_all(['p', 'h2'])]
        content_text = '\n'.join(filter(None, texts))
        
        data.append({"url": blog_url, "title": blog_title, "body": content_text})

In [20]:
with open("malaysia_property_reviews-EN.json", "w") as json_file:
    json.dump(data, json_file, indent=2)